# [DESARROLLO MATEMATICO] Seguidor solar

Dentro del presente desarrollo matematico, abordaremos dististos subtemas tales como:
- Rotacion compuesta

___
## ROTACIÓN COMPUESTA
Una rotación compuesta se define como la transformación resultante de aplicar sucesivamente dos o más rotaciones a un objeto o sistema de coordenadas.  Cada rotación individual se realiza alrededor de un eje específico, que puede ser diferente para cada rotación.
### Angulos de Euler
Una forma común de parametrizar las rotaciones es mediante los ángulos de Euler. Estos ángulos (φ, θ, ψ), que suelen denominarse roll, pitch y yaw,  representan tres rotaciones sucesivas alrededor de ejes específicos. La convención más habitual es la ZYX, que implica:

- Una rotación de ángulo ψ alrededor del eje Z.
- Una rotación de ángulo θ alrededor del eje Y'.
- Una rotación de ángulo φ alrededor del eje X''.

La representacion matematica de estos es mediante matrices, donde:
$$
R_{xyz}(\alpha, \beta, \gamma) = R_z(\gamma) R_y(\beta) R_x(\alpha) = \begin{bmatrix}
c\beta c\gamma & s\alpha s\beta c\gamma - c\alpha s\gamma & c\alpha s\beta c\gamma + s\alpha s\gamma \\
c\beta s\gamma & s\alpha s\beta s\gamma + c\alpha c\gamma & c\alpha s\beta s\gamma - s\alpha c\gamma \\
-s\beta & s\alpha c\beta & c\alpha c\beta
\end{bmatrix}
$$

Para facilitar los calculos posteriores, a la matriz '$R_{xyz}(\alpha, \beta, \gamma)$' la representaremos de la siguiente manera:
$$
R= \begin{bmatrix}
R_{11} & R_{12} & R_{13} \\
R_{21} & R_{22} & R_{23} \\
R_{31} & R_{32} & R_{33}
\end{bmatrix}
$$

Notación:

- cα, sα: representan cos(α) y sin(α), respectivamente (análogamente para β y γ).
- Rij: representa el elemento en la fila i y la columna j de la matriz de rotación R.

## CÁLCULO DE LOS ÁNGULOS DE EULER
Debido al proyecto que solicita calcular los dos angulos de control 'pitch' y 'roll' en base a la posicion solar 'azimuth' y 'elevacion'. Nos basaremos en el desarrollo matematico del siguinte video: https://www.youtube.com/watch?v=p6ZlxoHxjtQ&t=811s el cual nos facilita el calculo de los distintos angulos de Euler mediante la matriz 'R'. Donde:


### Cálculo de β (Pitch):


$$
\beta = \arctan\left(\frac{-R_{31}}{\pm\sqrt{R_{11}^2 + R_{21}^2}}\right)
$$

$$
\beta = \arctan2(-R_{31}, \sqrt{R_{11}^2 + R_{21}^2})
$$

donde:

$$
R_{31} = -s\beta
$$

$$
R_{11} = c\beta c\gamma
$$

$$
R_{21} = c\beta s\gamma
$$

### Cálculo de α (Roll):

$$
\alpha = \arctan\left(\frac{R_{32} / \cos(\beta)}{R_{33} / \cos(\beta)}\right)
$$

$$
\alpha = \arctan\left(\frac{R_{32}}{R_{33}}\right)
$$

$$
\alpha = \arctan2(R_{32}, R_{33})
$$

donde:

$$
R_{32} = s\alpha c\beta
$$

$$
R_{33} = c\alpha c\beta
$$

### Cálculo de γ (Yaw):

$$
\gamma = \arctan\left(\frac{R_{21} / \cos(\beta)}{R_{11} / \cos(\beta)}\right)
$$

$$
\gamma = \arctan\left(\frac{R_{21}}{R_{11}}\right)
$$

$$
\gamma = \arctan2(R_{21}, R_{11})
$$

donde:

$$
R_{21} = c\beta s\gamma
$$
$$
R_{11} = c\beta c\gamma
$$

## IMPLEMENTACIÓN
Debido que el panel solar del seguidor siempre debe estar posicionado de manera perpendicular a los rayos del sol. Se ha realizado una funcion llamada 'getSolarPosition' la cual es capaz de:
- Tomar una fecha, rango de horas, y una ubicación geográfica.
- Calcular el azimut y la altitud del sol en intervalos de 10 minutos.
- Calcular los ángulos adicionales (alpha: roll y beta: pitch) para orientar un panel solar.
- Devuelve los valores en listas organizadas.


In [20]:
from pysolar.solar import get_altitude, get_azimuth
import matplotlib.image as mpimg
from datetime import datetime, timedelta
from pytz import timezone
import numpy as np

def Rxyz(alpha, beta, gamma):
    """
    Calcula la matriz de rotación 3D Rxyz(α, β, γ).

    Args:
      alpha: Ángulo de rotación alrededor del eje X (en grados).
      beta: Ángulo de rotación alrededor del eje Y (en grados).
      gamma: Ángulo de rotación alrededor del eje Z (en grados).

    Returns:
      np.array: Matriz de rotación 3D.
    """
    # Convertir los ángulos a radianes
    alpha_rad = np.radians(alpha)
    beta_rad = np.radians(beta)
    gamma_rad = np.radians(gamma)

    # Matrices de rotación individuales
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(alpha_rad), -np.sin(alpha_rad)],
        [0, np.sin(alpha_rad), np.cos(alpha_rad)]
    ])

    Ry = np.array([
        [np.cos(beta_rad), 0, np.sin(beta_rad)],
        [0, 1, 0],
        [-np.sin(beta_rad), 0, np.cos(beta_rad)]
    ])

    Rz = np.array([
        [np.cos(gamma_rad), -np.sin(gamma_rad), 0],
        [np.sin(gamma_rad), np.cos(gamma_rad), 0],
        [0, 0, 1]
    ])

    # Multiplicar las matrices en el orden correcto: Rz(γ) * Ry(β) * Rx(α)
    R = np.dot(Rz, np.dot(Ry, Rx))
    return R

def getSolarPosition(
    start_date: datetime,
    start_hour: int = 6,
    end_hour: int = 18,
    latitude: float = -0.2105367,
    longitude: float = -78.491614
):
    """
    Calcula posiciones solares y ángulos para una fecha específica y rango de horas.

    Args:
      start_date (datetime): Fecha y hora de inicio para la simulación.
      start_hour (int): Hora de inicio del cálculo.
      end_hour (int): Hora de fin del cálculo.
      latitude (float): Latitud para la posición geográfica.
      longitude (float): Longitud para la posición geográfica.

    Returns:
      tuple: Tupla con los siguientes elementos:
          - times (list): Lista de tiempos de simulación.
          - azimuths (list): Lista de ángulos azimutales.
          - elevations (list): Lista de ángulos de elevación.
          - beta (list): Lista de ángulos de pitch.
          - alpha (list): Lista de ángulos de roll.
    """
    times = []
    azimuths = []
    elevations = []
    beta = []  # Pitch
    alpha = []  # Roll

    time_interval = timedelta(minutes=10)

    # Crear el rango de tiempos basado en la hora de inicio y fin
    start_time = start_date.replace(hour=start_hour, minute=0, second=0, microsecond=0)
    end_time = start_date.replace(hour=end_hour, minute=0, second=0, microsecond=0)

    current_time = start_time
    while current_time <= end_time:
        az = get_azimuth(latitude, longitude, current_time)
        el = get_altitude(latitude, longitude, current_time)

        # Convertir a radianes para los cálculos
        az_rad = np.radians(az)
        el_rad = np.radians(el)

        # --- Calcular beta (roll) y alpha (pitch) usando la matriz de rotación ---

        # 1. Calcular la matriz de rotación Rxyz(α, β, γ)
        gamma = 0            # (Asumimos gamma = 0 en este caso)
        # Calculamos beta y alpha
        val = np.cos(el_rad) * np.sin(az_rad)
        beta_rad_temp = np.arcsin(val)
        beta_deg_temp = np.degrees(beta_rad_temp)

        val_fi1 = -(np.cos(el_rad) * np.cos(az_rad)) / np.cos(beta_rad_temp)
        alpha_rad_temp = np.arcsin(val_fi1)
        alpha_deg_temp = np.degrees(alpha_rad_temp)

        R = Rxyz(alpha_deg_temp, beta_deg_temp, gamma)

        # 2. Obtener los elementos de la matriz
        R31 = R[2, 0]
        R11 = R[0, 0]
        R21 = R[1, 0]
        R32 = R[2, 1]
        R33 = R[2, 2]

        # 3. Calcular beta (pitch)
        beta_rad = np.arctan2(-R31, np.sqrt(R11**2 + R21**2))
        beta_deg = np.degrees(beta_rad)

        # 4. Calcular alpha (roll)
        alpha_rad = np.arctan2(R32, R33)
        alpha_deg = np.degrees(alpha_rad)

        # ---------------------------------------------------------------

        times.append(current_time)
        azimuths.append(az)
        elevations.append(el)
        beta.append(beta_deg)  # Pitch
        alpha.append(alpha_deg)  # Roll

        current_time += time_interval

    return times, azimuths, elevations, beta, alpha

In [21]:
# Ejemplo de uso
st_fecha = "2024-01-06"
start_date = datetime.strptime(st_fecha, "%Y-%m-%d")
start_date = start_date.replace(tzinfo=timezone("America/Guayaquil"))
print(start_date)

#start_date = datetime(2025, 1, 24, tzinfo=timezone("America/Guayaquil"))
times, azimuths, elevations, beta, phi = getSolarPosition(start_date)

# Imprimir los resultados
for i in range(len(times)):
    print(f"Tiempo: {times[i]}, Azimut: {azimuths[i]}, Elevación: {elevations[i]}, "
          f"Beta (pitch): {beta[i]}, Phi (roll): {phi[i]}")

2024-01-06 00:00:00-05:19
Tiempo: 2024-01-06 06:00:00-05:19, Azimut: 112.52505653487918, Elevación: 0.4397965696907528, Beta (pitch): 67.47087387699351, Phi (roll): 88.8520992021679
Tiempo: 2024-01-06 06:10:00-05:19, Azimut: 112.53364167783467, Elevación: 2.5279925524298594, Beta (pitch): 67.33234089144159, Phi (roll): 83.42807650103056
Tiempo: 2024-01-06 06:20:00-05:19, Azimut: 112.58094375571827, Elevación: 4.744094378948146, Beta (pitch): 66.95162859494982, Phi (roll): 77.80447449867523
Tiempo: 2024-01-06 06:30:00-05:19, Azimut: 112.66738393353847, Elevación: 7.004648311862432, Beta (pitch): 66.32963908242817, Phi (roll): 72.31670518365101
Tiempo: 2024-01-06 06:40:00-05:19, Azimut: 112.79369650119423, Elevación: 9.282543986876556, Beta (pitch): 65.48232995926612, Phi (roll): 67.12594428641376
Tiempo: 2024-01-06 06:50:00-05:19, Azimut: 112.96094193418996, Elevación: 11.567469965973569, Beta (pitch): 64.43143510144591, Phi (roll): 62.314972851383125
Tiempo: 2024-01-06 07:00:00-05:19, 

## INTERFAZ FECHA :)

In [22]:
import tkinter as tk
from tkinter import messagebox
from tkcalendar import Calendar
from datetime import datetime

# Variables globales para almacenar la fecha y las horas
st_fecha = None
start_t = None
end_t = None

def obtener_fecha_y_horas():
    global st_fecha, start_t, end_t  # Usar variables globales
    
    # Obtiene la fecha seleccionada
    fecha_seleccionada = calendario.get_date()
    st_fecha = datetime.strptime(fecha_seleccionada, "%m/%d/%y").strftime("%Y-%m-%d")
    
    # Obtiene las horas de inicio y fin
    hora_inicio = entry_inicio.get()
    hora_fin = entry_fin.get()
    
    try:
        start_t = datetime.strptime(hora_inicio, "%H").time()  
        end_t = datetime.strptime(hora_fin, "%H").time()
        
    
    except ValueError:
        messagebox.showerror("Error", "Por favor ingrese las horas en formato HH:MM")

# Crear la ventana principal
ventana = tk.Tk()
ventana.title("Selector de Fecha y Horas")
ventana.geometry("400x400")

# Crear un calendario
calendario = Calendar(ventana, selectmode="day", year=2025, month=1, day=27)
calendario.pack(pady=10)

# Etiqueta y campo de entrada para la hora de inicio - fin
label_inicio = tk.Label(ventana, text="Hora de inicio (HH):")
label_inicio.pack(pady=5)
entry_inicio = tk.Entry(ventana)
entry_inicio.pack(pady=5)

label_fin = tk.Label(ventana, text="Hora de fin (HH):")
label_fin.pack(pady=5)
entry_fin = tk.Entry(ventana)
entry_fin.pack(pady=5)

# Botón para obtener la fecha y horas seleccionadas
boton_seleccionar = tk.Button(ventana, text="Seleccionar Fecha y Horas", command=obtener_fecha_y_horas)
boton_seleccionar.pack(pady=10)

ventana.mainloop()

print("Fecha seleccionada:", st_fecha)
print("Hora de inicio:", start_t)
print("Hora de fin:", end_t)

Fecha seleccionada: 2023-01-04
Hora de inicio: 11:00:00
Hora de fin: 16:00:00


___
## POSICIÓN DEL PANEL SOLAR

In [23]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.animation as animation
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

def visualizar_trayectoria_panel(tiempos, azimuts, elevaciones, beta, phi):
    """
    Visualiza la trayectoria del panel solar en 3D.

    Args:
        tiempos (list): Lista de tiempos.
        azimuts (list): Lista de azimuts.
        elevaciones (list): Lista de elevaciones.
        beta (list): Lista de ángulos de pitch.
        phi (list): Lista de ángulos de roll.
    """

    # Configurar la ventana de Tkinter
    ventana_principal = tk.Tk()
    ventana_principal.title("Simulación del Movimiento del Panel Solar")

    # Crear un marco para la gráfica 3D
    marco_3d = tk.Frame(ventana_principal)
    marco_3d.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

    # Crear la figura y el eje 3D dentro de la ventana Tkinter
    figura = plt.Figure(figsize=(10, 7), dpi=100)
    ejes = figura.add_subplot(111, projection='3d')

    # Definir los parámetros del panel
    centro = [0, 0, 0]
    ancho = 4
    alto = 2

    # Función para crear los vértices del panel solar
    def construir_panel(centro, ancho, alto):
        """
        Crea los vértices del panel solar.

        Args:
            centro (list): Coordenadas del centro del panel.
            ancho (float): Ancho del panel.
            alto (float): Alto del panel.

        Returns:
            np.array: Array con las coordenadas de los vértices.
        """
        w = ancho / 2
        h = alto / 2
        vertices = np.array([
            [centro[0] - w, centro[1] - h, centro[2]],  # Inferior izquierdo
            [centro[0] + w, centro[1] - h, centro[2]],  # Inferior derecho
            [centro[0] + w, centro[1] + h, centro[2]],  # Superior derecho
            [centro[0] - w, centro[1] + h, centro[2]]   # Superior izquierdo
        ])
        return vertices

    # Crear los vértices del panel
    vertices = construir_panel(centro, ancho, alto)

   # Inicializar el panel con nuevos colores
    panel = Poly3DCollection([vertices], facecolors=['red', 'yellow'], linewidths=3, edgecolors='black')  # rojo y amarillo
    ejes.add_collection3d(panel)

    # Agregar marcadores en los vértices con nuevo color
    puntos_dispersion = [ejes.scatter(vertice[0], vertice[1], vertice[2], color='green', s=50) for vertice in vertices]  # verde

    # Configurar los límites de la gráfica
    ejes.set_xlim(-5, 5)
    ejes.set_ylim(-5, 5)
    ejes.set_zlim(-5, 5)

    # Marcar la dirección del Norte (eje Y positivo)
    ejes.quiver(0, 0, 0, 0, 5, 0, color='blue', linewidth=2)  # Flecha apuntando al norte 
    ejes.text(0, 5, 0, "Norte", color='blue', fontsize=12)      # Etiqueta de "Norte"


    # Añadir texto para mostrar el tiempo
    etiqueta_tiempo = ejes.text2D(0.05, 0.95, "", transform=ejes.transAxes, color='black', fontsize=12)


    # Función para aplicar ambas rotaciones (pitch y roll) usando matrices combinadas
    def aplicar_rotaciones(vertices, angulo_pitch, angulo_roll):
        """
        Aplica las rotaciones de pitch y roll a los vértices.
        """
        # Usar la función Rxyz para calcular la matriz de rotación combinada
        rotacion_combinada = Rxyz(angulo_roll, angulo_pitch, 0)  # gamma = 0

        # Aplicar la rotación combinada a los vértices
        return vertices @ rotacion_combinada.T

    # Función para inicializar la animación
    def iniciar_animacion():
        """
        Inicializa la animación.
        """
        panel.set_verts([vertices])
        etiqueta_tiempo.set_text('')  # Iniciar con texto vacío
        for punto in puntos_dispersion:
            punto._offsets3d = (vertices[:, 0], vertices[:, 1], vertices[:, 2])
            return [panel] + puntos_dispersion + [etiqueta_tiempo]

    # Función para actualizar la animación con los valores de `beta` y `phi`
    def actualizar_animacion(fotograma):
        """
        Actualiza la animación en cada fotograma.

        Args:
            fotograma (int): Número de fotograma actual.
        """
        if fotograma < len(tiempos):
            # Aplicar rotaciones combinadas
            vertices_rotados = aplicar_rotaciones(vertices, beta[fotograma], phi[fotograma])

            # Actualizar las posiciones del panel
            panel.set_verts([vertices_rotados])

            # Actualizar las posiciones de los puntos en los vértices
            for i, punto in enumerate(puntos_dispersion):
                punto._offsets3d = (np.array([vertices_rotados[i, 0]]),
                                        np.array([vertices_rotados[i, 1]]),
                                        np.array([vertices_rotados[i, 2]]))

            # Actualizar la etiqueta de tiempo con el tiempo calculado
            tiempo_actual = tiempos[fotograma].strftime("Hora: %Hh%M")
            etiqueta_tiempo.set_text(tiempo_actual)
        
        return [panel] + puntos_dispersion + [etiqueta_tiempo]

    # Crear el lienzo para la figura 3D
    canvas3 = FigureCanvasTkAgg(figura, master=marco_3d)
    canvas3.draw()
    canvas3.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    # Crear la animación dentro de la ventana Tkinter, sin blit
    intervalo_tiempo_ms = 1000  # Un frame por segundo para sincronizar con el tiempo real
    animacion = animation.FuncAnimation(figura, actualizar_animacion, frames=np.arange(0, len(tiempos)), init_func=iniciar_animacion, interval=intervalo_tiempo_ms, blit=False)

    # Iniciar la interfaz gráfica de Tkinter
    ventana_principal.mainloop()

In [24]:
# Ejemplo de uso

start_date = datetime.strptime(st_fecha, "%Y-%m-%d")
start_date = start_date.replace(tzinfo=timezone("America/Guayaquil"))
print(start_date)

times, azimuths, elevations, beta, alpha = getSolarPosition(start_date, start_hour=start_t.hour, end_hour=end_t.hour)

# Llamar a la función para graficar la trayectoria
visualizar_trayectoria_panel(times, azimuths, elevations, beta, alpha)

2023-01-04 00:00:00-05:19


___
## TAYECTORIA SOLAR